In [1]:
import time
import os
import numpy as np
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # disable infos
os.environ["TF_ENABLE_ONEDNN_OPTS"]="1" # make sure onednn is used on CPU
# tf.config.set_visible_devices([], 'GPU') # GPU improves c-times much

T=3.0
N=int(T*2*12)
Nsim = 10
# N=int(T*4*12) # no real difference
# Nsim = 5
r=0.0303
dtype=np.float32
seed=1
tf.random.set_seed(seed)

t=np.linspace(0,T,N*Nsim,endpoint=True,dtype=dtype).reshape((-1,1))
t=tf.constant(t)

from Commodities import Schwartz2Factor
'Salmon'
# mu, sigma1, sigma2, kappa, alpha, lambda, rho, delta0, P0
# salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.01, 0.9, 0.57, 95] # down,down
salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.2, 0.9, 0.57, 95] # down,up
# salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.6, 0.9, 0.57, 95] # up,up

'Soy'
# mu, sigma1, sigma2, kappa, alpha, lambda, rho, delta0, P0
# soyParam=[0.15, 0.5, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # low vol
soyParam=[0.15, 1, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # medium vol
# soyParam=[0.15, 2, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # high vol

'Risk neutral dynamics'
salmonParam[0]=r
soyParam[0]=r

"Fish feeding 25% of production cost, disease 30%, harvest 10%. Total production cost = 50% of price = labor, smolt, ..."
salmonPrice=salmonParam[-1] #NOK/KG
productionCosts=salmonPrice*0.5
harvestingCosts=productionCosts*0.1 # roughly 10%
feedingCosts=productionCosts*0.25
biologicalCosts=productionCosts*0.3
initialSalmon=(salmonPrice-productionCosts)+feedingCosts+harvestingCosts+biologicalCosts #we add the costs to salmon price since they are respected in the model, other costs are fixed and thus removed
salmonParam[-1]=initialSalmon
print(f'Feeding costs {feedingCosts} and Harvesting costs {harvestingCosts}')


soy=Schwartz2Factor(soyParam,t,dtype=dtype)
salmon=Schwartz2Factor(salmonParam,t,dtype=dtype)


from Harvest import Harvest
hc = harvestingCosts
harvest = Harvest(hc)

from Growth import Bertalanffy
wInf=6
a=1.113
b=1.097
c=1.43
growth = Bertalanffy(t,wInf,a,b,c)

from Price import Price
price = Price(salmon)

from Feed import StochFeed,DetermFeed
cr=1.1
fc=feedingCosts
feed_s = StochFeed(fc,cr,r,t,soy)
feed_d = DetermFeed(fc,cr,r,t,soy)

from Mortality import ConstMortatlity,HostParasite,DetermHostParasite,Poisson,DetermPoisson
# n0=10000
# m=0.1
# mort = ConstMortatlity(t,n0,m) # not comparable to the other models, because of neglected treatment costs

params=[0.05,0.1,7.01,0.05]
beta=[0.0829,0.0281]
H0=10000.0
P0=10
treatmentCosts=0.015 #0.015 is baseline
tData=np.array([0.018868,0.037736,0.056604,0.075472,0.09434,0.11321,0.13208,0.15094,0.16981,0.18868,0.20755,0.22642,0.24528,0.26415,0.28302,0.30189,0.32075,0.33962,0.35849,0.37736,0.39623,0.41509,0.43396,0.45283,0.4717,0.49057,0.50943,0.5283,0.54717,0.56604,0.58491,0.60377,0.62264,0.64151,0.66038,0.67925,0.69811,0.71698,0.73585,0.75472,0.77358,0.79245,0.81132,0.83019,0.84906,0.86792,0.88679,0.90566,0.92453,0.9434,0.96226,0.98113,1,1.0189,1.0377,1.0566,1.0755,1.0943,1.1132,1.1321,1.1509,1.1698,1.1887,1.2075,1.2453,1.2642,1.283,1.3019,1.3208,1.3396,1.3585,1.3774,1.3962,1.4151,1.434,1.4528,1.4717,1.4906,1.5094,1.5283,1.5472,1.566,1.5849,1.6038,1.6226,1.6415,1.6604,1.6792,1.6981,1.717,1.7358,1.7547,1.7736
],dtype=np.float32).reshape((-1,))
dm=np.array([0.0062441,0.0099877,0.015766,0.024537,0.037613,0.056736,0.084146,0.12261,0.1754,0.24623,0.33908,0.45793,0.60649,0.78782,1.004,1.2559,1.5429,1.8628,2.2121,2.5861,2.9792,3.3853,3.798,4.2111,4.6192,5.017,5.4006,5.7665,6.1123,6.4363,6.7376,7.0159,7.2713,7.5045,7.7163,7.908,8.0807,8.2359,8.3748,8.4988,8.6094,8.7077,8.795,8.8723,8.9407,9.0012,9.0546,9.1017,9.1432,9.1798,9.212,9.2403,9.2652,9.287,9.3062,9.3231,9.3379,9.3509,9.3623,9.3723,9.3811,9.3888,9.3956,9.4067,9.4112,9.4152,9.4187,9.4218,9.4245,9.4268,9.4289,9.4307,9.4323,9.4337,9.4349,9.436,9.4369,9.4377,9.4384,9.4391,9.4396,9.4401,9.4405,9.4409,9.4412,9.4415,9.4417,9.442,9.4421,9.4423,9.4425,9.4426
],dtype=np.float32).reshape((-1,))

mort_s = HostParasite(t,params,beta,H0,P0,treatmentCosts=treatmentCosts)
mort_d = DetermHostParasite(t,params,beta,H0,P0,treatmentCosts=treatmentCosts)

# mort_s = Poisson(H0,params[0],t,tData[1:],dm,treatmentCosts=treatmentCosts)
# mort_d = DetermPoisson(H0,params[0],t,tData[1:],dm,treatmentCosts=treatmentCosts)

from FishFarm import fishFarm
farm_ss = fishFarm(growth,feed_s,price,harvest,mort_s,stride=Nsim,seed=seed)
farm_sd = fishFarm(growth,feed_s,price,harvest,mort_d,stride=Nsim,seed=seed)
farm_ds = fishFarm(growth,feed_d,price,harvest,mort_s,stride=Nsim,seed=seed)
farm_dd = fishFarm(growth,feed_d,price,harvest,mort_d,stride=Nsim,seed=seed)

from OptimalStopping import Polynomial,DeepOptS,LSMC
batch_size=2**12 #need to fix it for simplicity
batches=20 # not so relevant since we make pathwise comparison, only relevant for value of opt stopping

Feeding costs 11.875 and Harvesting costs 4.75


c:\ProgramData\Anaconda3\envs\intelpython310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
farm_ss.seed(seed)
tf.random.set_seed(seed)
X_ss,V_ss,Vh_ss,ft_ss = farm_ss.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_ss=np.array(X_ss)
V_ss=np.array(V_ss)
Vh_ss=np.array(Vh_ss)
ft_ss=np.array(ft_ss)
print(X_ss.shape)

(73, 81920, 6)


In [3]:
farm_sd.seed(seed)
tf.random.set_seed(seed)
X_sd,V_sd,Vh_sd,ft_sd = farm_sd.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_sd=np.array(X_sd)
V_sd=np.array(V_sd)
Vh_sd=np.array(Vh_sd)
ft_sd=np.array(ft_sd)
print(X_sd.shape)

(73, 81920, 4)


In [4]:
farm_ds.seed(seed)
tf.random.set_seed(seed)
X_ds,V_ds,Vh_ds,ft_ds = farm_ds.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_ds=np.array(X_ds)
V_ds=np.array(V_ds)
Vh_ds=np.array(Vh_ds)
ft_ds=np.array(ft_ds)
print(X_ds.shape)

(73, 81920, 4)


In [5]:
farm_dd.seed(seed)
tf.random.set_seed(seed)
X_dd,V_dd,Vh_dd,ft_dd = farm_dd.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_dd=np.array(X_dd)
V_dd=np.array(V_dd)
Vh_dd=np.array(Vh_dd)
ft_dd=np.array(ft_dd)
print(X_dd.shape)

(73, 81920, 2)


In [6]:
batches=20
farm_ss.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_ss=DeepOptS(r,farm_ss.tCoarse,farm_ss.generateFishFarm,d=farm_ss.d,batch_size=batch_size)
optDeep_ss.train(batches)

tic=time.time()
tauDOS_ss,VtauDOS_ss=optDeep_ss.evaluate(X_ss,V_ss,Vh_ss,ft_ss)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_ss)} with mean value {np.mean(VtauDOS_ss)} in {ctimeEval} s')


100%|██████████| 1506/1506 [01:22<00:00, 18.15epoch/s, loss=-2.49e+6, payoff=2.52e+6]


Mean stopping time 1.9883369207382202 with mean value 2501648.5 in 0.7885222434997559 s


In [7]:
batches=20
farm_sd.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_sd=DeepOptS(r,farm_sd.tCoarse,farm_sd.generateFishFarm,d=farm_sd.d,batch_size=batch_size)
optDeep_sd.train(batches)

tic=time.time()
tauDOS_sd,VtauDOS_sd=optDeep_sd.evaluate(X_sd,V_sd,Vh_sd,ft_sd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_sd)} with mean value {np.mean(VtauDOS_sd)} in {ctimeEval} s')


100%|██████████| 1504/1504 [00:45<00:00, 32.92epoch/s, loss=-2.44e+6, payoff=2.47e+6]


Mean stopping time 1.9926927089691162 with mean value 2461948.0 in 0.6856491565704346 s


In [8]:
batches=20
farm_ds.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_ds=DeepOptS(r,farm_ds.tCoarse,farm_ds.generateFishFarm,d=farm_ds.d,batch_size=batch_size)
optDeep_ds.train(batches)

tic=time.time()
tauDOS_ds,VtauDOS_ds=optDeep_ds.evaluate(X_ds,V_ds,Vh_ds,ft_ds)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_ds)} with mean value {np.mean(VtauDOS_ds)} in {ctimeEval} s')


100%|██████████| 1504/1504 [00:45<00:00, 33.18epoch/s, loss=-2.42e+6, payoff=2.45e+6]


Mean stopping time 2.0060033798217773 with mean value 2442247.75 in 0.6889743804931641 s


In [9]:
batches=20
farm_dd.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_dd=DeepOptS(r,farm_dd.tCoarse,farm_dd.generateFishFarm,d=farm_dd.d,batch_size=batch_size)
optDeep_dd.train(batches)

tic=time.time()
tauDOS_dd,VtauDOS_dd=optDeep_dd.evaluate(X_dd,V_dd,Vh_dd,ft_dd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_dd)} with mean value {np.mean(VtauDOS_dd)} in {ctimeEval} s')


100%|██████████| 1502/1502 [00:42<00:00, 35.04epoch/s, loss=-2.38e+6, payoff=2.41e+6]


Mean stopping time 2.0051918029785156 with mean value 2401190.25 in 0.5753045082092285 s


In [10]:
print('Stoch Feeding and Stoch Mortality as baseline')
tComp,Vcomp = fishFarm.compareStoppingTimes(V_ss,[tauDOS_ss,tauDOS_sd,tauDOS_ds,tauDOS_dd],np.array(farm_ss.tCoarse))
for i in range(0,len(tComp)):
    print(f'Mean tau: {tComp[i]} with mean {Vcomp[i]} with ratio {Vcomp[0]/Vcomp[i]}')

Stoch Feeding and Stoch Mortality as baseline
Mean tau: [1.98858566] with mean 2501648.6610439555 with ratio 1.0
Mean tau: [1.99294052] with mean 2466925.7420805916 with ratio 1.0140753806938991
Mean tau: [2.00627022] with mean 2441185.4388775853 with ratio 1.0247679759199981
Mean tau: [2.00545406] with mean 2405409.703965217 with ratio 1.0400093825680061


In [11]:
print('Determ Feeding and Stoch Mortality as baseline')
tComp,Vcomp = fishFarm.compareStoppingTimes(V_ds,[tauDOS_ds,tauDOS_dd],np.array(farm_ss.tCoarse))
for i in range(0,len(tComp)):
    print(f'Mean tau: {tComp[i]} with mean {Vcomp[i]} with ratio {Vcomp[0]/Vcomp[i]}')

Determ Feeding and Stoch Mortality as baseline
Mean tau: [2.00627022] with mean 2442247.9001732706 with ratio 1.0
Mean tau: [2.00545406] with mean 2406204.572636294 with ratio 1.0149793280034733


In [12]:
basis_ss = Polynomial(deg=2,dtype=dtype)
basis_sd = Polynomial(deg=2,dtype=dtype)
basis_ds = Polynomial(deg=2,dtype=dtype)
basis_dd = Polynomial(deg=2,dtype=dtype)
optLSMC_ss=LSMC(r,farm_ss.tCoarse,farm_ss.generateFishFarm,basis_ss,batch_size=batch_size)
optLSMC_sd=LSMC(r,farm_sd.tCoarse,farm_sd.generateFishFarm,basis_sd,batch_size=batch_size)
optLSMC_ds=LSMC(r,farm_ds.tCoarse,farm_ds.generateFishFarm,basis_ds,batch_size=batch_size)
optLSMC_dd=LSMC(r,farm_dd.tCoarse,farm_dd.generateFishFarm,basis_dd,batch_size=batch_size)

tic=time.time()
tau_ss,Vtau_ss=optLSMC_ss.evaluate(X_ss,V_ss,Vh_ss,ft_ss)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_ss)} with mean value {np.mean(Vtau_ss)} in {ctimeEval} s')

tic=time.time()
tau_sd,Vtau_sd=optLSMC_sd.evaluate(X_sd,V_sd,Vh_sd,ft_sd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_sd)} with mean value {np.mean(Vtau_sd)} in {ctimeEval} s')

tic=time.time()
tau_ds,Vtau_ds=optLSMC_ds.evaluate(X_ds,V_ds,Vh_ds,ft_ds)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_ds)} with mean value {np.mean(Vtau_ds)} in {ctimeEval} s')

tic=time.time()
tau_dd,Vtau_dd=optLSMC_dd.evaluate(X_dd,V_dd,Vh_dd,ft_dd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_dd)} with mean value {np.mean(Vtau_dd)} in {ctimeEval} s')

Mean stopping time 1.9049365520477295 with mean value 2443017.0 in 5.742878198623657 s
Mean stopping time 2.015138626098633 with mean value 2468648.75 in 1.8985505104064941 s
Mean stopping time 1.906715750694275 with mean value 2384695.25 in 1.942720651626587 s
Mean stopping time 2.032078504562378 with mean value 2413580.25 in 1.8925843238830566 s


In [13]:
print('Stoch Feeding and Stoch Mortality as baseline')
tComp,Vcomp = fishFarm.compareStoppingTimes(V_ss,[tau_ss,tau_sd,tau_ds,tau_dd],np.array(farm_ss.tCoarse))
for i in range(0,len(tComp)):
    print(f'Mean tau: {tComp[i]} with mean {Vcomp[i]} with ratio {Vcomp[0]/Vcomp[i]}')

Stoch Feeding and Stoch Mortality as baseline
Mean tau: 1.904936600133442 with mean 2443017.139156914 with ratio 1.0
Mean tau: 2.0151385885730635 with mean 2473601.830551052 with ratio 0.9876355640522287
Mean tau: 1.9067157030534871 with mean 2383846.765370178 with ratio 1.024821383088165
Mean tau: 2.0320785182819234 with mean 2417877.950963974 with ratio 1.0103972113989117


In [14]:
print('Determ Feeding and Stoch Mortality as baseline')
tComp,Vcomp = fishFarm.compareStoppingTimes(V_ds,[tau_ds,tau_dd],np.array(farm_ss.tCoarse))
for i in range(0,len(tComp)):
    print(f'Mean tau: {tComp[i]} with mean {Vcomp[i]} with ratio {Vcomp[0]/Vcomp[i]}')

Determ Feeding and Stoch Mortality as baseline
Mean tau: 1.9067157030534871 with mean 2384695.1739433287 with ratio 1.0
Mean tau: 2.0320785182819234 with mean 2418526.8492393494 with ratio 0.9860114534983719
